In [15]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from scipy import stats
from IPython.display import display

def export(df: pd.DataFrame, path_no_dot: str, index=True):
    df.to_csv(f'exports/{path_no_dot}.csv', index=index)
    df.to_excel(f'exports/{path_no_dot}.xlsx', index=index)

In [16]:
# data initialization

# needs ppt_analysis.ipynb to be ran first
players_table = pd.read_csv('exports/players_table.csv')
matches_table = pd.read_csv('exports/matches_table.csv').sort_values(['event', 'match_number']).reset_index(drop=True)
matches_table.drop('Unnamed: 0', axis=1, inplace=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2,seed1,seed2,tpc,D_seeds,adjD_seeds,seed1_favor,seed2_favor,D_scores,adjD_scores,upset,abs_upset
0,2021_2s,1,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3,1,8,6,-7,-0.875000,93.750000,6.250000,-3,-0.500000,0.500000,0.500000
1,2021_2s,2,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3,4,5,6,-1,-0.125000,56.250000,43.750000,-3,-0.500000,-0.357143,0.357143
2,2021_2s,3,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3,2,7,6,-5,-0.625000,81.250000,18.750000,-3,-0.500000,0.214286,0.214286
3,2021_2s,4,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0,3,6,6,-3,-0.375000,68.750000,31.250000,-6,-1.000000,-0.571429,0.571429
4,2021_2s,5,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6,8,5,6,3,0.375000,31.250000,68.750000,2,0.333333,-0.095238,0.095238
5,2021_2s,6,True,True,Ann Hall,Anna Brown,Paul Bartenfeld,Leah Baetcke,6,4,7,6,6,1,0.125000,43.750000,56.250000,-2,-0.333333,-0.476190,0.476190
6,2021_2s,7,False,False,Rose Roché,Aaron Carter,Coby Lovelace,Jason Jackson,5,6,1,4,6,-3,-0.375000,68.750000,31.250000,1,0.166667,0.595238,0.595238
7,2021_2s,8,False,False,Kevin Cooper,Roman Ramirez,Rohan Chowla,Levin Lee,5,6,2,3,6,-1,-0.125000,56.250000,43.750000,1,0.166667,0.309524,0.309524
8,2021_2s,9,True,True,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,5,2,7,6,-5,-0.625000,81.250000,18.750000,-1,-0.166667,0.547619,0.547619
9,2021_2s,10,True,True,Rose Roché,Aaron Carter,Jack Massingill,Will Simpson,5,6,1,5,6,-4,-0.500000,75.000000,25.000000,1,0.166667,0.738095,0.738095


In [17]:
# a player with a 400 point advantage over their opponent is 
# ten times more like to win than to lose

# the k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, but with few games, we want to change it frequently

# we want the function to vary on the y-axis from 6 to -6

def expected_score(ratingA, ratingB): # probability of winning
    return 1 / (1 + np.power(10, (ratingB - ratingA) / (400)))

def expected_diff(ratingA, ratingB): # expected amount of cups to be scored (version 2)
    pass

def rating_change(score, expected_score):
    K = 80
    return K * (score - expected_score)

def rating_change_2(score, expected_score, cup_diff):
    K = 40
    # cup_diff_mults = [0.75, 1, 1.25, 1.5, 1.75, 2]
    cup_diff_mults = [0.5, 1, 1.5, 2, 2.5, 3]
    # return K * (score - expected_score) * (cup_diff)
    return K * (score - expected_score) * (cup_diff_mults[cup_diff - 1])

# ELO INITIALIZATION
starting_elo = 1200.0
elo = dict()
elo_time = np.zeros([len(players_table['player']), len(matches_table) + 1])
for player in players_table['player']:
    elo.update({player: starting_elo})
    elo_time[players_table['player'][players_table['player'] == player].index[0], 0] = starting_elo


# ALGORITHM
for (i, row) in matches_table.iterrows():
    elos_t1 = list()
    elos_t2 = list()

    elos_t1.append(elo[row.t1p1])
    elos_t2.append(elo[row.t2p1])

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):
        elos_t1.append(elo[row.t1p2])
        elos_t2.append(elo[row.t2p2])

    elo_t1 = np.mean(elos_t1)
    elo_t2 = np.mean(elos_t2)

    win_prob_t1 = expected_score(elo_t1, elo_t2)
    win_prob_t2 = expected_score(elo_t2, elo_t1)

    t1_won = row.score1 > row.score2

    # elo[row.t1p1] += rating_change(t1_won, win_prob_t1)
    # elo[row.t2p1] += rating_change(not t1_won, win_prob_t2)

    # 1.0 means multiply your change by 1.0 if you win
    win_inflation_rate = 1.0
    # means multiply your change by 0.3 if you lose
    # penalty for losing is lesser by 70%
    loss_inflation_rate = 1.0

    if t1_won:
        t1_inflation = win_inflation_rate
        t2_inflation = loss_inflation_rate
    else:
        t1_inflation = loss_inflation_rate
        t2_inflation = win_inflation_rate


    elo[row.t1p1] += rating_change_2(t1_won, win_prob_t1, abs(row.score1 - row.score2)) * t1_inflation
    elo[row.t2p1] += rating_change_2(not t1_won, win_prob_t2, abs(row.score1 - row.score2)) * t2_inflation


    elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = elo[row.t1p1]
    elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = elo[row.t2p1]

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):
        # elo[row.t1p2] += rating_change(t1_won, win_prob_t1)
        # elo[row.t2p2] += rating_change(not t1_won, win_prob_t2)
        elo[row.t1p2] += rating_change_2(t1_won, win_prob_t1, abs(row.score1 - row.score2)) * t1_inflation
        elo[row.t2p2] += rating_change_2(not t1_won, win_prob_t2, abs(row.score1 - row.score2)) * t2_inflation
        
        elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = elo[row.t1p2]
        elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = elo[row.t2p2]

players_table['current_elo'] = players_table['player'].map(elo)

player_elo_table = players_table[['player', 'current_elo']].sort_values('current_elo', ascending=False).reset_index(drop=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(player_elo_table)

export(player_elo_table, 'player_elo_table', False)

,player,current_elo
0,Levin Lee,1331.880857
1,Roman Ramirez,1326.793759
2,Aaron Carter,1315.152064
3,Rohan Chowla,1259.021623
4,Kevin Cooper,1248.171436
5,Will Simpson,1226.764427
6,Nathan Snow,1221.183102
7,Gabe Silverstein,1218.575477
8,Coby Lovelace,1216.925597
9,Jack Massingill,1212.620330


In [18]:
elo_time_table = pd.concat([players_table.player, pd.DataFrame(elo_time)], axis=1).T
elo_time_table.columns = players_table.player
elo_time_table = elo_time_table.iloc[2:,:]
elo_time_table.replace(0.0, np.nan, inplace=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(elo_time_table)

export(elo_time_table, 'elo_time_table', True)

player,Aaron Carter,Ann Hall,Anna Brown,Brian Tafazoli,Carla Betancourt,Cason Duszak,Cassie Deering,Coby Lovelace,Evan Sooklal,Gabe Silverstein,Helen Dunn,Jack Massingill,Jason Jackson,Kevin Cooper,Kristian Banlaoi,Leah Baetcke,Levin Lee,Matthew Rusten,Nathan Snow,Noah Dale,Paul Bartenfeld,Piper Parker,Reagan Fryatt,Rohan Chowla,Roman Ramirez,Rose Roché,Sam Carswell-Tellis,Will Simpson,Yvonne Nguyen
1,1222.500000,NaN,NaN,1177.5,NaN,NaN,NaN,NaN,1177.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1222.500000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1222.500000,NaN,NaN,NaN,1177.500000,1222.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1177.500000,NaN
3,NaN,1177.500000,1177.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1222.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1222.500000,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1155.000000,1245.000000,NaN,NaN,NaN,1155.000000,NaN,NaN,1245.000000,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,1162.5,NaN,NaN,NaN,NaN,1162.500000,NaN,NaN,1192.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1192.500000,NaN
6,NaN,1191.529953,1191.529953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1140.970047,NaN,NaN,NaN,NaN,1140.970047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1215.000000,NaN,NaN,NaN,NaN,NaN,NaN,1230.000000,NaN,NaN,NaN,NaN,1230.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1215.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1215.485024,NaN,NaN,1252.014976,NaN,NaN,NaN,NaN,NaN,NaN,1252.014976,1215.485024,NaN,NaN,NaN,NaN
9,NaN,1184.546247,1184.546247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1222.468730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1222.468730,NaN,NaN,NaN,NaN
10,1207.014976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200.485024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1207.014976,NaN,1200.485024,NaN


In [19]:
import plotly.graph_objects as go


fig = go.Figure()

for (i,player) in enumerate(players_table.player):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        mode='lines+markers',
        line_shape='hv',
        name = player,
        connectgaps=True,
        # hoverinfo='x+y'
    ))

fig.add_vrect(
    annotation_text="2021 2s",
    annotation_position="top left",
    x0=1,
    x1=15,
    fillcolor='green',
    opacity=0.1,
    line_width=0,
)

fig.add_vrect(
    annotation_text="2022 1s",
    annotation_position="top left",
    x0=15,
    x1=43,
    fillcolor='red',
    opacity=0.1,
    line_width=0,
)

# 2022 doubles
fig.add_vrect(
    annotation_text="2022 2s",
    annotation_position="top left",
    x0=43,
    x1=60,
    fillcolor='yellow',
    opacity=0.1,
    line_width=0,
)

# 2023 singles
fig.add_vrect(
    annotation_text="2023 1s",
    annotation_position="top left",
    x0=60,
    x1=97,
    fillcolor='blue',
    opacity=0.1,
    line_width=0,
)


fig.update_layout(
    title='Pong ELO over Time',
    xaxis_title='Game Number',
    yaxis_title='ELO',
    # autosize=False,
    # width=800,
    # height=800
)

fig.show()
fig.write_html("exports/elo_interactive_graph.html")

In [20]:
expected_score(1924, 1232)

0.9817195269701807

In [21]:

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2,seed1,seed2,tpc,D_seeds,adjD_seeds,seed1_favor,seed2_favor,D_scores,adjD_scores,upset,abs_upset
0,2021_2s,1,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3,1,8,6,-7,-0.875000,93.750000,6.250000,-3,-0.500000,0.500000,0.500000
1,2021_2s,2,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3,4,5,6,-1,-0.125000,56.250000,43.750000,-3,-0.500000,-0.357143,0.357143
2,2021_2s,3,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3,2,7,6,-5,-0.625000,81.250000,18.750000,-3,-0.500000,0.214286,0.214286
3,2021_2s,4,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0,3,6,6,-3,-0.375000,68.750000,31.250000,-6,-1.000000,-0.571429,0.571429
4,2021_2s,5,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6,8,5,6,3,0.375000,31.250000,68.750000,2,0.333333,-0.095238,0.095238
5,2021_2s,6,True,True,Ann Hall,Anna Brown,Paul Bartenfeld,Leah Baetcke,6,4,7,6,6,1,0.125000,43.750000,56.250000,-2,-0.333333,-0.476190,0.476190
6,2021_2s,7,False,False,Rose Roché,Aaron Carter,Coby Lovelace,Jason Jackson,5,6,1,4,6,-3,-0.375000,68.750000,31.250000,1,0.166667,0.595238,0.595238
7,2021_2s,8,False,False,Kevin Cooper,Roman Ramirez,Rohan Chowla,Levin Lee,5,6,2,3,6,-1,-0.125000,56.250000,43.750000,1,0.166667,0.309524,0.309524
8,2021_2s,9,True,True,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,5,2,7,6,-5,-0.625000,81.250000,18.750000,-1,-0.166667,0.547619,0.547619
9,2021_2s,10,True,True,Rose Roché,Aaron Carter,Jack Massingill,Will Simpson,5,6,1,5,6,-4,-0.500000,75.000000,25.000000,1,0.166667,0.738095,0.738095
